<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-2-public/blob/adding_C4/C4/W2/ungraded_labs/C4_W2_Lab_2_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Setup

In [1]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from google.colab import drive
tfds.disable_progress_bar()

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.12.0


## Download the Dataset

In [11]:
splits = ['train[:60%]', 'train[-40%:]', 'test']

splits, info = tfds.load(name='imdb_reviews', split=splits, as_supervised=True, with_info=True)

train_data, validation_data, test_data = splits

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Explore the Data [IMDB DATASET]

The labels are either 0 or 1, where 0 is a negative review, and 1 is a positive review. We will create a list with the corresponding class names, so that we can map labels to class names later on.

In [14]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples
num_classes = info.features['label'].num_classes

print('The Dataset has a total of:')
print('\u2022 {:,} classes'.format(num_classes))

print('\u2022 {:,} movie reviews for training'.format(num_train_examples))
print('\u2022 {:,} movie reviews for testing'.format(num_test_examples))

The Dataset has a total of:
• 2 classes
• 25,000 movie reviews for training
• 25,000 movie reviews for testing


In [15]:
print(train_data)

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [17]:
class_names = ['negative', 'positive']

Each example consists of a sentence representing the movie review and a corresponding label. The sentence is not preprocessed in any way. Let's take a look at the first example of the training set.  

In [18]:
for review, label in train_data.take(1):
    review = review.numpy()
    label = label.numpy()
    print('\nMovie Review:\n\n', review)
    print('\nLabel:', class_names[label])


Movie Review:

 b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

Label: negative


# Load Data [Primary Dataset]

In [3]:
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/knowze/sentiment')

Mounted at /content/drive


In [4]:
# Load csv file

dataset = pd.read_csv('data_sentimenv2.csv')

# Cleaning Data
dataset['text'] = dataset['text'].str.lower()  # Ubah ke lowercase
dataset['text'] = dataset['text'].str.replace('[^\w\s]', '')  # Hapus karakter khusus

# Turn Text into list
sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# for item in range(0,5):
#   label = labels[item]
#   print(sentences[item] + ' Labels : ')
#   print(label)

# Turn Text Into tensor
sentences_tensor = tf.constant(np.array(sentences))
print(sentences_tensor[0])

tf.Tensor(b'kursus ini sangat membantu dan informatif materi disajikan dengan baik', shape=(), dtype=string)


<ipython-input-4-3341c6e05c72>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['text'] = dataset['text'].str.replace('[^\w\s]', '')  # Hapus karakter khusus


In [5]:
# Persiapan label agar sesuai dengan panjang teks setelah padding
padded_labels = []
for label in labels:
    # padded_label = label[:max_length]  # Ambil sebagian label sesuai dengan panjang teks yang sudah dipad
    padded_labels.append(label)

padded_labels = tf.constant(padded_labels)

# Tampilkan hasil preprocessing
print(padded_labels)

tf.Tensor(
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1
 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0
 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0
 0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 1
 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1

In [6]:
max_data_length = tf.size(labels).numpy()
max_length= 20
max_data_length

590

## Load Word Embeddings

Pake model yang otomatis pre-processing sentences nya

In [7]:
# if you are running the notebook on Colab
# embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
embedding = "https://www.kaggle.com/models/google/nnlm/frameworks/TensorFlow2/variations/id-dim50/versions/1"

# # if you are running the notebook on your local machine
# embedding = "./models/tf2-preview_gnews-swivel-20dim_1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string ,trainable=False)

## Build Pipeline

In [31]:
# batch_size = 512

# train_batches = train_data.shuffle(num_train_examples // 4).batch(batch_size).prefetch(1)
# validation_batches = validation_data.batch(batch_size).prefetch(1)
# test_batches = test_data.batch(batch_size)

In [8]:
# Shuffle Data
combined_data = tf.data.Dataset.from_tensor_slices((sentences_tensor, padded_labels))
# print(list(combined_data.as_numpy_iterator())[0])
shuffled_data = combined_data.shuffle(buffer_size=len(sentences_tensor))

# Define training and validation sets
training_set = shuffled_data.take(500)
validation_set = shuffled_data.skip(500)

# Configure the batch size and prefetch for better performance
batch_size = 32
training_set = training_set.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_set = validation_set.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [9]:
# Mengonversi dataset TensorFlow menjadi iterator Numpy
training_iterator = iter(training_set)
# Mendapatkan beberapa contoh dari set pelatihan
num_examples = 1  # Ganti dengan jumlah contoh yang ingin Anda lihat
for i in range(num_examples):
    example_data = next(training_iterator)
    print("Contoh Data:", i + 1)
    print("Input Data:", example_data[0])  # Cetak input data
    print("Label Data:", example_data[1])  # Cetak label data

Contoh Data: 1
Input Data: tf.Tensor(
[b'belajar memainkan alat musik yang sulit diatur'
 b'materinya oke lah tapi kurang greget'
 b'belajar menulis cerita dengan plot yang tak jelas'
 b'belajar memasak dengan resep yang teruji'
 b'belajar menyanyi dengan nada yang tidak pas'
 b'isinya keren sangat informatif'
 b'materinya lumayan sih tapi kayak kurang aja gitu'
 b'materi sangat terlalu pendek kurang menjelaskan dengan baik'
 b'saya senang menemukan kursus ini materi sangat bermanfaat'
 b'belajar menyusun presentasi yang menarik'
 b'gak masuk akal gak memberikan manfaat' b'gak bermanfaat banget aneh'
 b'kursusnya soso aja sih kurang menarik'
 b'materi sangat detail sangat memperkaya wawasan' b'belajar membuat bom'
 b'pengembangnya hebat hasilnya luar biasa'
 b'kursusnya lumayan sih tapi materinya agak membingungkan'
 b'kontennya kurang greget sih menurutku kurang update gitu'
 b'saya sangat puas dengan kedalaman materi yang disampaikan'
 b'cara mengatur keuangan untuk masa depan'
 b're

## Build the Model

In [10]:
model = tf.keras.Sequential([
        hub_layer,
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])

## Train the Model

Since this is a binary classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use the `binary_crossentropy` loss function.

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(training_set,
                    epochs=40,
                    validation_data=validation_set)

Epoch 1/40
16/16 [==============================] - 1s 16ms/step - loss: 0.6845 - accuracy: 0.5780 - val_loss: 0.6695 - val_accuracy: 0.6222
Epoch 2/40
16/16 [==============================] - 0s 3ms/step - loss: 0.6621 - accuracy: 0.6520 - val_loss: 0.6405 - val_accuracy: 0.7333
Epoch 3/40
16/16 [==============================] - 0s 3ms/step - loss: 0.6343 - accuracy: 0.7120 - val_loss: 0.6095 - val_accuracy: 0.7778
Epoch 4/40
16/16 [==============================] - 0s 3ms/step - loss: 0.5961 - accuracy: 0.7420 - val_loss: 0.5562 - val_accuracy: 0.8111
Epoch 5/40
16/16 [==============================] - 0s 3ms/step - loss: 0.5620 - accuracy: 0.7760 - val_loss: 0.5148 - val_accuracy: 0.8222
Epoch 6/40
16/16 [==============================] - 0s 3ms/step - loss: 0.5132 - accuracy: 0.8260 - val_loss: 0.4929 - val_accuracy: 0.8111
Epoch 7/40
16/16 [==============================] - 0s 3ms/step - loss: 0.4664 - accuracy: 0.8380 - val_loss: 0.4709 - val_accuracy: 0.8000
Epoch 8/40
16/16 [=

## Evaluate the Model

We will now see how well our model performs on the testing set.

In [12]:
eval_results = model.evaluate(validation_set, verbose=0)

for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.3}'.format(value))

loss: 0.146
accuracy: 0.967


In [15]:
my_array = [["Cara membakar rumah orang yang kita benci"], ["Cara mulai gym"], ["Cara membunuh orang dan menyembunyikannya"]]
print(my_array)

prediction = []
def predict(list_sentences):
  for sentence in list_sentences:
    predict  = model.predict(sentence)
    prediction.append(predict)
  for i in prediction:
    print(i)
    if i[0] > 0.5:
      print('Positif')
    else:
      print('Negatif')

predict(my_array)
# print(prediction)



[['Cara membakar rumah orang yang kita benci'], ['Cara mulai gym'], ['Cara membunuh orang dan menyembunyikannya']]
1/1 [==============================] - 0s 41ms/step
[[0.00068896]]
Negatif
[[0.96464574]]
Positif
[[0.48389444]]
Negatif


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Teks baru yang ingin Anda prediksi
new_text = ["Fuck your movie is so bad"]

# Inisialisasi CountVectorizer untuk tokenisasi
vectorizer = CountVectorizer()

# Lakukan tokenisasi pada teks
vectorizer.fit(new_text)

# Ubah teks menjadi representasi vektor
text_vector = vectorizer.transform(new_text)

# Output hasil tokenisasi
print("Vocabulary:")
print(vectorizer.vocabulary_)  # Tampilkan daftar kata-kata yang dihasilkan sebagai token

print("\nVectorized Text:")
print(text_vector.toarray())  # Tampilkan representasi vektor dari teks berdasarkan token


Vocabulary:
{'fuck': 1, 'your': 5, 'movie': 3, 'is': 2, 'so': 4, 'bad': 0}

Vectorized Text:
[[1 1 1 1 1 1]]
